In [ ]:
import pandas as pd
import numpy as np
import json
import os

## Run export_SQL.sh

In [ ]:
data_dir = '~/Documents/Python/graphLearning/mTurk-10-trans-prob/'

In [ ]:
participants = pd.read_csv("".join([data_dir, 'experiment/data/raw/participants_test.csv.gz']))
participants.head(n=10)

In [ ]:
np.sum(participants['workerid'] == "A2EFENZUAL6Z9V")

In [ ]:
participants = participants[participants['workerid'] == "A2EFENZUAL6Z9V"]
participants.info()

In [ ]:
experiment = pd.read_csv("".join([data_dir, '/experiment/data/raw/experiment_test.csv.gz']))
experiment.head(n=10)

In [ ]:
walkdata = pd.read_csv("".join([data_dir, '/experiment/data/raw/walkdata_test.csv.gz']))
walkdata.head(n=10)

In [ ]:
datastring = \
    participants.\
    loc[~participants.datastring.isnull()].\
    datastring.\
    apply(json.loads).\
    apply(pd.Series)

In [ ]:
import zlib
import base64
from json import loads
def decompress_pako(datastring):
    """
    Decompress json data that we compressed in the browser with paco.
    Assumes data was then base64-encoded:
    btoa(pako.deflate(JSON.stringify(data), { to: 'string' }));
    Parameters
    ----------
    datastring : string
        base64-encoded json data to decompress
    Returns
    -------
    dict
        JSON-decoded and decompressed data
    """
    data = loads(zlib.decompress(base64.b64decode(datastring)))
    return data

In [ ]:
taskdata_list = []
for row in datastring.itertuples(index=False):
    if 'compressed_task_data' in row.questiondata:
        data = pd.DataFrame(decompress_pako(row.questiondata['compressed_task_data']))
        data['workerid'] = row.workerId
        data['assignmentid'] = row.assignmentId
        data['uniqueid'] = data.workerid + ':' + data.assignmentid
        taskdata_list.append(data)
taskdata = pd.concat(taskdata_list).reset_index(drop=True)
taskdata.tail(n=10)

In [ ]:
taskdata[taskdata['event'] == 'correct'][taskdata['keyCode'] == 32]